In [29]:
# import os
import re
import pymupdf
import pandas as pd

In [4]:
file_path = "handbook.pdf"

In [3]:
print(pymupdf.__version__)

1.26.3


In [6]:
from tqdm.auto import tqdm

c:\Users\Stephanie\Downloads\LLM-Zoomcamp-Capstone-Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def read_pdf(file_path):
    doc = pymupdf.open(file_path)
    pages_and_texts = []

    for page_number , page in enumerate(tqdm(doc, desc="Reading Pages....")):
        text = page.get_text()
        # text = text.replace("\n", " ").strip()
        pages_and_texts.append({
            "page number": page_number - 11,
            "number of characters": len(text),
            "word count": len(text.split(" ")),
            "page_token_count": len(text) / 4,
            "text": text
        })

    return pages_and_texts

pages_and_texts = read_pdf(file_path)
    

Reading Pages....: 100%|██████████| 192/192 [00:00<00:00, 197.57it/s]


In [21]:
print(pages_and_texts[13])

{'page number': 2, 'number of characters': 1420, 'word count': 238, 'page_token_count': 355.0, 'text': '2  value for their investment. It is also common knowledge that every  covenant is ratified by blood and, as a church-sponsored  University, we consider the blood of Jesus Christ, which is the  blood of the everlasting covenant, as our stronghold in the  fulfilment of this awesome obligation. Covenant University vows  to make of her graduates, expert thinkers, leader-managers, and  hyper-resourceful technocrats in all fields of human endeavour.  1.2.  THE UNIVERSITY  On October 21, 2002, the African educational landscape was  radically altered by the formal entry of Covenant University (CU)  into the higher education context. The University is located at  Canaanland, Ota, Ogun State, Nigeria. The University is a growing,  dynamic, vision-birthed and vision-driven University, founded on  a Christian mission ethos and committed to pioneering excellence  at the cutting edge of learning.

In [26]:
df = pd.DataFrame(pages_and_texts)
df.head()

,page number,number of characters,word count,page_token_count,text
0,-11,0,1,0.00,
1,-10,62,19,15.50,COVENANT UNIVERSITY STUDENT HANDBOOK ...
2,-9,335,83,83.75,i Covenant University Student Handbook | 201...
3,-8,482,105,120.50,"ii CHANCELLOR David O. Oyedepo, Ph.D., FNAE ..."
4,-7,2438,188,609.50,iii TABLE OF CONTENTS Table of Contents .....


In [27]:
# Get stats
df.describe().round(2)

,page number,number of characters,word count,page_token_count
count,192.00,192.00,192.00,192.00
mean,84.50,1101.18,185.34,275.30
std,55.57,481.36,60.11,120.34
min,-11.00,0.00,1.00,0.00
25%,36.75,886.75,156.75,221.69
50%,84.50,1113.50,203.00,278.38
75%,132.25,1321.25,227.00,330.31
max,180.00,3051.00,291.00,762.75


In [ ]:
# mean token count = 275.30
# so I can use a 384 embedding model!

I want to split each page into sections and subsections

In [81]:
pattern = re.compile(
    r"(\d+(?:\.\d+)*)(?:\.\s*|\s+)([A-Z][A-Z\s:\-\(\)]+?)(?=(?:\s+[A-Z][a-z])|(?:\s*\d+\.)|$)"
)

def split_sections(text):
    matches = list(pattern.finditer(text))
    sections = []
    
    for i, match in enumerate(matches):
        title = match.string[match.start():match.end()].strip()
        start = match.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(text)
        content = text[start:end].strip()
        
        sections.append({
            "title": title,
            "text": content
        })
    return sections

In [86]:
document = []
current_chapter = ""
for page in pages_and_texts[11:]:
    sections = split_sections(page["text"])
    if sections != []:
        if sections["text"] == "":
            current_chapter = sections["title"]

        sections['chapter'] = current_chapter
        document.append(sections)

TypeError: list indices must be integers or slices, not str

In [85]:
document

[]